#### Simulating the travelling salesman problem with three cities.

* Cost Matrix

In [1]:
K = [0, 2, 3, 4, 0, 6, 7, 8, 0]

* Importing important libraries

In [96]:
from qiskit import *
from qiskit.circuit import ParameterVector, Parameter
from qiskit.visualization import plot_histogram
from IPython.core.display import Latex
from qiskit_algorithms.optimizers import ADAM
from qiskit_aer import AerSimulator, StatevectorSimulator
import numpy as np
import ot
import re
from qiskit.circuit.library.standard_gates import RXGate, RYGate, RZGate

* Number of Cities 'n'
* Quantum circuit 
* Applying Hadammard to all qubits

In [3]:
n = 3   
qc = QuantumCircuit(n*n)
qc.h([i for i in range(n*n)])     # creating equal superposition

* Parameters

In [4]:
beta_para = (2 * np.pi * np.random.rand(1))               # Beta parameters
theta1 = (2 * np.pi * np.random.rand(1)).tolist()       # Perceptron layers 1st parameters
theta2 = (2 * np.pi * np.random.rand(1)).tolist()       # Perceptron layers 2nd parameters
alpha = (2 * np.pi * np.random.rand(1)).tolist()         # Alpha parameters for pooling layer

* Preparing the multi-controlled X rotational gate

In [99]:
MCRX=RXGate(beta_para[0]).control(4, ctrl_state='0000')

In [103]:
class TSP_Solver:
    def __init__(self, qc, n, K, theta1, theta2, alpha, matching_matrix):
        self.qc = qc
        self.n = n
        self.n1 = n*n  # Squared of number of cities
        self.K = K
        self.alpha = alpha[0]
        self.theta1 = theta1[0]
        self.theta2 = theta2[0]
        self.matching_matrix = matching_matrix  # A constraint matrix for matching with the ground truth
    
    "Creating an object of encoding layer"
    def encoding_layer(self):
        for q in range(len(self.K)):
            self.qc.ry(self.K[q], q)
        self.qc.barrier()
        return self.qc
            
            
    "Defining contraint layer"
    def constraint_layer(self): 
        self.qc.append(MCRX, [1, 2, 3, 6, 0])    # rotation on qbit_1
        self.qc.append(MCRX, [0, 2, 4, 7, 1])    # rotation on qbit_2
        self.qc.append(MCRX, [0, 1, 5, 8, 2])    # rotation on qbit_3
        self.qc.append(MCRX, [0, 6, 4, 5, 3])    # rotation on qbit_4
        self.qc.append(MCRX, [1, 7, 3, 5, 4])    # rotation on qbit_5
        self.qc.append(MCRX, [2, 3, 4, 8, 5])    # rotation on qbit_6
        self.qc.append(MCRX, [0, 3, 7, 8, 6])    # rotation on qbit_7
        self.qc.append(MCRX, [1, 4, 6, 8, 7])    # rotation on qbit_8
        self.qc.append(MCRX, [6, 7, 2, 5, 8])    # rotation on qbit_9
        self.qc.barrier()
    
    "Perceptron layer"
    def perceptron_layer(self):
        rz = RZGate(self.theta1)
        ry = RYGate(self.theta2)
        for q in range(n1):
            self.qc.append(rz, [q])
            self.qc.append(ry, [q])
        self.qc.barrier()   
               
    'Pooling Layer'
    def pooling_layer(self):
        ry = RYGate(self.alpha)
        for q in range(n1):
            self.qc.append(ry, [q])
        
    'Measurement layer'
    def measurement_layer(self):
        self.qc.measure_all()
    
    'Statvector simulating'
    def statevect(self):
        simulator = StatevectorSimulator()
        result= simulator.run(transpile(self.qc, simulator), shots = 1000).result()
        statevector = result.get_statevector()
        return statevector.draw('latex')
    
    'Reshaping result to matrix'
    def x_matrix(self):
        latex_string = self.statevect().data     # Stores the statevector as a latex string5
        latex_list = list(latex_string)        # This object stores the statevector as a list
        
        'Extracting only the states of the qubits'
        a = latex_list.index('|')
        b = latex_list.index('\\')
        y = latex_list[a+1:b]
        y1 = np.array([int(i) for i in y])
        Y1 = y1.reshape(self.n, self.n)  # Reshaping the measured states of qubits to a matrix 
        
        # Define the size of marginals
        n = Y1.shape[0]
        m = Y1.shape[1]
        
        # Define the cost matrix with the matching constraint
        cost_matrix = np.multiply(Y1, self.matching_matrix) # Element-wise multiplication to zero out non-matching elements

        # Apply Sinkhorn algorithm to converge to a doubly stochastic matrix with the matching constraint
        X_ds = ot.sinkhorn(np.ones(n) / n, np.ones(m) / m, cost_matrix, reg=0.01)
        return X_ds
    
    'Creating the cost function as cross-entrophy loss'
    def cost_fun(self):
        'Objective function'
        def objective_func():
            """Cost function of circuit parameters on training data.
            The optimizer will attempt to minimize this."""
            return self.cost_fun()
            loss = 0
            X = list(self.matching_matrix.flatten())
            Y = list(self.x_matrix().flatten())
            for i in range(len(X)):
                loss += (-X[i]*np.log(Y[i]) + (1-X[i])*np.log(1-Y[i]))
            return loss
        return objective_func
    


#### Calling the whole class here

In [106]:
def call_pass_obj_fun(TSP):
    TSP.encoding_layer()
    TSP.constraint_layer()
    TSP.perceptron_layer()
    TSP.pooling_layer()
    TSP.measurement_layer()
    TSP.statevect()
    TSP.x_matrix()
    obj = TSP.cost_fun()
    return obj
    
    
# Define the matching constraint
m_c = np.array([[0, 1, 0],
                [1, 0, 0],
                [0, 0, 1]])
n = 3
qc = QuantumCircuit(n*n)
qc.h([i for i in range(n*n)])     # creating equal superposition    
TSP = TSP_Solver(qc, n, K, theta1, theta2, alpha, m_c)

<function __main__.TSP_Solver.cost_fun.<locals>.objective_func()>

###### A main function from where the whole class or TSP-Simulation be controlled

In [108]:
# Main Function
def main():
    
    class optimizerlog:
        """Log to store optimizer's intermediate results"""
        def __init__(self):
            self.evaluations = []
            self.parameters = []
            self.costs = []
        def update(self, evaluation, parameter, cost, _stepsize, _accept):
            """Save intermediate results. Optimizer passes five values
            but we ignore the last two."""
            self.evaluations.append(evaluation)
            self.parameters.append(parameter)
            self.costs.append(cost)

    log = optimizerlog()
    
    # Creating the adam optimizer and minimizing the cross-entropy function
    adam = ADAM(maxiter=100, tol=1e-06, lr=0.01, beta_1=0.9, beta_2=0.99)
    result = adam.minimize(27, call_pass_obj_fun(TSP))

if __name__ == '__main__':
    main()

TypeError: 'int' object is not callable

In [109]:
!git add .
!git commit -m "Your commit message"

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> pyconcorde
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached pyconcorde
hint: 
hint: See "git help submodule" for more information.


[main f9c36f7] Your commit message
 4 files changed, 410 insertions(+), 106 deletions(-)
 create mode 100644 .ipynb_checkpoints/Not_MCRX-checkpoint.ipynb
 create mode 160000 pyconcorde


In [ ]:
!git push -u origin main

In [110]:
!git rm --cached pyconcorde

rm 'pyconcorde'
